In [2]:
"""
SageMath verification code for (permutation) and (APN) results

This code verifies:
- Theorem \ref{thm:main_result_sec3}: G_a is permutation <-> P_a(T) has no roots
- Theorem \ref{thm:APN_result}: G_a is APN <-> P_a(T) has no roots
- Corollary \ref{cor:perm_APN_equiv}: G_a is permutation <-> G_a is APN
"""

def test_permutation(m, i, a):
    """
    Test if G_a is a permutation over F_{2^m}^3.
    
    Returns: (is_permutation, image_size, domain_size)
    """
    F = GF(2^m, 'w')
    q = 2^i
    
    image_set = set()
    
    for x in F:
        for y in F:
            for z in F:
                comp1 = x^(q+1) + a * x^q * z + y * z^q
                comp2 = x^q * z + y^(q+1)
                comp3 = x * y^q + a * y^q * z + z^(q+1)
                
                image = (comp1, comp2, comp3)
                image_set.add(image)
    
    domain_size = F.cardinality()^3
    image_size = len(image_set)
    
    return (image_size == domain_size, image_size, domain_size)


def compute_differential_uniformity(m, i, a):
    """
    Compute the differential uniformity of G_a.
    
    Returns: (delta, is_APN) where delta is the differential uniformity
             and is_APN is True if delta == 2
    """
    F = GF(2^m, 'w')
    q = 2^i
    
    max_solutions = 0
    
    # Iterate over all nonzero directions
    for alpha in F:
        for beta in F:
            for gamma in F:
                if alpha == 0 and beta == 0 and gamma == 0:
                    continue
                
                # For each direction, iterate over all targets
                for u in F:
                    for v in F:
                        for w in F:
                            solution_count = 0
                            
                            # Count solutions to the differential equation
                            for x in F:
                                for y in F:
                                    for z in F:
                                        # Compute G_a(x,y,z)
                                        g1 = x^(q+1) + a * x^q * z + y * z^q
                                        g2 = x^q * z + y^(q+1)
                                        g3 = x * y^q + a * y^q * z + z^(q+1)
                                        
                                        # Compute G_a(x+alpha, y+beta, z+gamma)
                                        x2, y2, z2 = x + alpha, y + beta, z + gamma
                                        g1_shifted = x2^(q+1) + a * x2^q * z2 + y2 * z2^q
                                        g2_shifted = x2^q * z2 + y2^(q+1)
                                        g3_shifted = x2 * y2^q + a * y2^q * z2 + z2^(q+1)
                                        
                                        # Check if difference equals target
                                        if (g1_shifted + g1 == u and 
                                            g2_shifted + g2 == v and 
                                            g3_shifted + g3 == w):
                                            solution_count += 1
                            
                            max_solutions = max(max_solutions, solution_count)
    
    return (max_solutions, max_solutions == 2)


def has_roots_Pa(F, q, a):
    """
    Check if P_a(T) = T^{q^2+q+1} + (a*T^q + 1)^(q+1) has roots in F.
    
    Returns: (has_roots, list_of_roots)
    """
    exponent = q^2 + q + 1
    roots = []
    for t in F:
        term1 = t^exponent
        term2 = (a * t^q + 1)^(q+1)
        if term1 + term2 == 0:
            roots.append(t)
    return (len(roots) > 0, roots)


def comprehensive_verification(m, i):
    """
    Verification for given (m, i).
    Checks polynomial condition, permutation property, and APN property.
    """
    if gcd(i, m) != 1:
        print(f"Warning: gcd({i}, {m}) != 1")
        return None
    
    if (m // gcd(i, m)) % 2 == 0:
        print(f"Warning: m/gcd(i,m) = {m//gcd(i,m)} is even")
        return None
    
    F = GF(2^m, 'w')
    q = 2^i
    
    print(f"\n{'='*80}")
    print(f"VERIFICATION FOR m={m}, i={i}, q={q}")
    print(f"Field: F_{{2^{m}}} with {F.cardinality()} elements")
    print(f"Domain: F_{{2^{m}}}^3 with {F.cardinality()^3} elements")
    print(f"Condition: m/gcd(i,m) = {m//gcd(i,m)} (odd)")
    print(f"Polynomial: P_a(T) = T^{q^2+q+1} + (a*T^q + 1)^(q+1)")
    print(f"{'='*80}")
    
    results = {
        'permutations': [],
        'APN_functions': [],
        'APN_permutations': [],
        'non_permutations': [],
        'non_APN': [],
        'correlation_matches': 0,
        'correlation_failures': []
    }
    
    total_tested = 0
    
    for a in F:
        if a == 0:
            continue
        
        total_tested += 1
        print(f"\n{'='*60}")
        print(f"Testing a = {a} ({total_tested}/{F.cardinality()-1})")
        print(f"{'='*60}")
        
        # Step 1: Check polynomial condition
        print("Step 1: Checking if P_a(T) has roots...")
        has_roots, roots = has_roots_Pa(F, q, a) 
        print(f" P_a(T) has roots: {has_roots}")
        if has_roots:
            print(f" Roots: {roots}")
        
        # Step 2: Check permutation property
        print("\nStep 2: Checking permutation property...")
        is_perm, img_size, dom_size = test_permutation(m, i, a)
        print(f" Is permutation: {is_perm}")
        if not is_perm:
            print(f" Image size: {img_size}/{dom_size}")
        
        # Step 3: Check APN property
        print("\nStep 3: Computing differential uniformity...")
        print(" (This may take a while for larger fields...)")
        delta, is_APN = compute_differential_uniformity(m, i, a)
        print(f" Differential uniformity δ = {delta}")
        print(f" Is APN (δ=2): {is_APN}")
        
        # Step 4: Check correlations
        print("\nStep 4: Checking correlations...")
        predicted_perm = not has_roots
        predicted_APN = not has_roots
        
        perm_match = (predicted_perm == is_perm)
        APN_match = (predicted_APN == is_APN)
        
        print(f" Polynomial condition predicts permutation: {predicted_perm}")
        print(f" Actual permutation: {is_perm}")
        print(f" Match: {perm_match}")
        print(f" Polynomial condition predicts APN: {predicted_APN}")
        print(f" Actual APN: {is_APN}")
        print(f" Match: {APN_match}")
        
        if perm_match and APN_match:
            results['correlation_matches'] += 1
            print("  CORRELATION VERIFIED")
        else:
            results['correlation_failures'].append({
                'a': a,
                'has_roots': has_roots,
                'predicted_perm': predicted_perm,
                'actual_perm': is_perm,
                'predicted_APN': predicted_APN,
                'actual_APN': is_APN,
                'delta': delta
            })
            print("  CORRELATION FAILED")
        
        # Record results
        if is_perm:
            results['permutations'].append({
                'a': a,
                'Pa_has_roots': has_roots,
                'delta': delta,
                'is_APN': is_APN
            })
        else:
            results['non_permutations'].append({
                'a': a,
                'Pa_has_roots': has_roots,
                'image_size': img_size,
                'delta': delta,
                'is_APN': is_APN
            })
        
        if is_APN:
            results['APN_functions'].append({
                'a': a,
                'is_perm': is_perm,
                'Pa_has_roots': has_roots
            })
        else:
            results['non_APN'].append({
                'a': a,
                'delta': delta,
                'is_perm': is_perm
            })
        
        if is_perm and is_APN:
            results['APN_permutations'].append(a)
    
    # Print summary
    print(f"\n{'='*80}")
    print(f"RESULTS SUMMARY")
    print(f"{'='*80}")
    print(f"Total non-zero a values tested: {total_tested}")
    print(f"Permutations found: {len(results['permutations'])}")
    print(f"APN functions found: {len(results['APN_functions'])}")
    print(f"APN permutations found: {len(results['APN_permutations'])}")
    print(f"Correlation matches: {results['correlation_matches']}/{total_tested}")
    
    correlation_rate = 100.0 * results['correlation_matches'] / total_tested
    print(f"Correlation rate: {correlation_rate:.1f}%")
    
    if results['correlation_matches'] == total_tested:
        print(f"\n THEOREMS VERIFIED:")
        print(f"  • Theorem~\\ref{{thm:main_result_sec3}}: G_a is permutation <-> P_a(T) has no roots")
        print(f"  • Theorem~\\ref{{thm:APN_result}}: G_a is APN <-> P_a(T) has no roots")
        print(f"  • Corollary~\\ref{{cor:perm_APN_equiv}}: G_a is permutation <-> G_a is APN")
    else:
        print(f"\n✗ CORRELATION FAILURES DETECTED:")
        for failure in results['correlation_failures']:
            print(f"  a={failure['a']}:")
            print(f"    Predicted perm={failure['predicted_perm']}, actual={failure['actual_perm']}")
            print(f"    Predicted APN={failure['predicted_APN']}, actual={failure['actual_APN']}")
            print(f"    δ={failure['delta']}")
    
    return results


def run_all_verifications():
    """
    Run verifications for all test cases.
    """
    test_cases = [
        (3, 1),  # m=3, i=1, q=2
        #(3, 2),  # m=3, i=2, q=4
        #(5, 1),  # m=5, i=1, q=2 (warning: this will take a long time!)
    ]
    
    print("="*80)
    print("COMPREHENSIVE VERIFICATION OF THEOREMS on PP and APN")
    print("="*80)
    
    all_results = {}
    
    for m, i in test_cases:
        try:
            results = comprehensive_verification(m, i)
            all_results[(m, i)] = results
        except KeyboardInterrupt:
            print(f"\nSkipping (m={m}, i={i}) - computation interrupted")
            continue
        except Exception as e:
            print(f"\nError for (m={m}, i={i}): {e}")
            import traceback
            traceback.print_exc()
            continue
    
    # Final summary
    print("\n" + "="*80)
    print("FINAL SUMMARY ACROSS ALL TEST CASES")
    print("="*80)
    
    for (m, i), results in all_results.items():
        if results is None:
            continue
        total = results['correlation_matches'] + len(results['correlation_failures'])
        if total > 0:
            rate = 100.0 * results['correlation_matches'] / total
            status = " VERIFIED" if rate == 100.0 else " FAILED"
            print(f"(m={m}, i={i}): {results['correlation_matches']}/{total} "
                  f"({rate:.1f}%) {status}")
            print(f"  APN permutations: {len(results['APN_permutations'])}")
    
    return all_results


# Run the verification
if __name__ == "__main__":
    results = run_all_verifications()

COMPREHENSIVE VERIFICATION OF THEOREMS on PP and APN

VERIFICATION FOR m=3, i=1, q=2
Field: F_{2^3} with 8 elements
Domain: F_{2^3}^3 with 512 elements
Condition: m/gcd(i,m) = 3 (odd)
Polynomial: P_a(T) = T^7 + (a*T^q + 1)^(q+1)

Testing a = w (1/7)
Step 1: Checking if P_a(T) has roots...
 P_a(T) has roots: False

Step 2: Checking permutation property...
 Is permutation: True

Step 3: Computing differential uniformity...
 (This may take a while for larger fields...)
 Differential uniformity δ = 2
 Is APN (δ=2): True

Step 4: Checking correlations...
 Polynomial condition predicts permutation: True
 Actual permutation: True
 Match: True
 Polynomial condition predicts APN: True
 Actual APN: True
 Match: True
  CORRELATION VERIFIED

Testing a = w^2 (2/7)
Step 1: Checking if P_a(T) has roots...
 P_a(T) has roots: False

Step 2: Checking permutation property...
 Is permutation: True

Step 3: Computing differential uniformity...
 (This may take a while for larger fields...)
 Differential uni